In [17]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False
from elasticsearch import Elasticsearch
import json


import configparser

from incremental_learning.config import root_dir

In [3]:
config = configparser.ConfigParser()

config.read(root_dir / 'config.ini')

['/home/valeriy/Documents/workspace/valeriy42/ml-cpp/jupyter/config.ini']

In [4]:
es = Elasticsearch(cloud_id=config['cloud']['cloud_id'],
                  http_auth=(config['cloud']['user'], config['cloud']['password']))

In [5]:
from pathlib2 import Path

In [20]:
paths = list(root_dir.glob('**/experiment_data/*'))

In [21]:
paths

[PosixPath('/home/valeriy/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/prototypes/experiment_data/1'),
 PosixPath('/home/valeriy/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/prototypes/experiment_data/_sources'),
 PosixPath('/home/valeriy/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/prototypes/experiment_data/3'),
 PosixPath('/home/valeriy/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/prototypes/experiment_data/2')]

In [22]:
for path in paths:
    if not (path/'config.json').is_file():
        print('{} ignored'.format(path)
        continue 
        
    doc = {}
    with open(path/'config.json') as fp:
        config = json.load(fp)
        doc['config'] = config

    with open(path/'metrics.json') as fp:
        metrics = json.load(fp)
        doc['metrics'] = metrics
        
    with open(path/'run.json') as fp:
        run = json.load(fp)
        doc['run'] = run
        artifacts = run['artifacts']
        run['artifacts'] = []
        for artifact in artifacts:
            with open(path/artifact) as fp:
                run['artifacts'].append({'name': artifact, 'value': fp.read()})
        
    with open(path/'cout.txt') as fp:
        cout = fp.read()
        doc['cout'] = cout

    res = es.index(index='experiments', body=doc)
    print('{} {}'.format(path, res['result']))

/home/valeriy/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/prototypes/experiment_data/1 created
/home/valeriy/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/prototypes/experiment_data/_sources
/home/valeriy/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/prototypes/experiment_data/3 created
/home/valeriy/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/prototypes/experiment_data/2 created


In [24]:
from pathlib2 import Path

In [25]:
p = Path('/home/valeriy/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/')
for d in p.iterdir():
    print(d)

/home/valeriy/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/__init__.py
/home/valeriy/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/incremental_learning
/home/valeriy/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/archive
/home/valeriy/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/.ipynb_checkpoints
/home/valeriy/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/prototypes


In [5]:
import eland as ed

In [8]:
comments = ed.DataFrame(es_client=es, es_index_pattern="experiment-multi-step-sampling", 
             columns=['experiment_uid', 'run.meta.comment'], )

In [13]:
df = comments.to_pandas()

In [16]:
df = df.dropna()

In [37]:
for row in df.iterrows():
    uid = row[1].experiment_uid
    comment = row[1]['run.meta.comment']
    q = {
         "script": {
            "inline": "ctx._source.comment='{}'".format(comment),
            "lang": "painless"
         },
         "query": {
            "match": {
                "experiment_uid": uid
            }
         }
    }

    es.update_by_query(body=q, index='experiment-multi-step-sampling-metrics')